In [1]:
from subprocess import Popen, PIPE, CalledProcessError
import re
import platform
import shutil
import os.path

options = {'stdout': PIPE, 'stderr': PIPE, 'bufsize' : 1, 'universal_newlines' : True, 'shell' : False}
if (platform.system() == 'Linux'):
    #options['shell'] = True
    #Packages required to generate ssh keys in windows
    from cryptography.hazmat.primitives import serialization as crypto_serialization
    from cryptography.hazmat.primitives.asymmetric import rsa
    from cryptography.hazmat.backends import default_backend as crypto_default_backend

def callPopen(cmd):
    print(cmd)
    with Popen(cmd.split(),**options) as p:
        for line in p.stdout:
            print(line, end='')
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)

def isInstance(name):
    instanceExists=False
    ip=''
    with Popen('gcloud compute instances list'.split(),**options) as p:
        for line in p.stdout:
            if re.match('^{}'.format(name), line):
                instanceExists=True
                ip = line.strip().split()
                ip = ip[4]
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)
        return(instanceExists,ip)
    
            
def text_prepender(filename, text):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(text.rstrip('\r\n') + content)
        
if (platform.system() == 'Windows'):
    def generateSSHKey(username,savePath):
        key = rsa.generate_private_key(
            backend=crypto_default_backend(),
            public_exponent=65537,
            key_size=2048
            )
        private_key = key.private_bytes(
            crypto_serialization.Encoding.PEM,
            crypto_serialization.PrivateFormat.TraditionalOpenSSL,
            crypto_serialization.NoEncryption()
            )
        public_key = key.public_key().public_bytes(
            crypto_serialization.Encoding.OpenSSH,
            crypto_serialization.PublicFormat.OpenSSH
            )
        public_file = os.path.join(savePath,username + '.pub')
        private_file = os.path.join(savePath,username)
        text_file = open(public_file, "w")
        text_file.write(public_key.decode('utf-8') + ' ' + username)
        text_file.close()
        text_file = open(private_file, "w")
        text_file.write(private_key.decode('utf-8'))
        text_file.close()
        print('Successfully created key pair')
            
if (platform.system() == 'Linux'):
    def generateSSHKey(username,savePath):
        p = Popen("echo 'yes' | ssh-keygen -t rsa -f {0}/{1} -C {1} -N '' ".format(savePath,username),
              stdout=PIPE,
              shell=True,
              stderr=PIPE
               )
        print(p.communicate())    

# Automated calibration for the GOTM/FABM model at Langtjern

## Pre-requisites

* gcloud command line utility
* google cloud account, you might need to be an admin for some of this to work, also this is not free
* ssh key setup in gitlab.au.dk
* fabric3 needs to be (pip) installed
* python modules listed in the first cell of this notebook

## Creating an instance on Google Cloud

In [2]:
instanceName = 'gotm-cal'
username = 'jose-luis'

createInstance = '''\
gcloud compute instances create {} \
--zone europe-west3-a \
--image-family debian-9 \
--image-project debian-cloud \
--machine-type n1-standard-2 \
'''
#--boot-disk-size 200GB \

deleteInstance = '''\
gcloud compute instances delete {} \
--zone europe-west-3a \
'''

listInstances = '''gcloud compute instances list'''

addSSHKeys = '''gcloud compute instances add-metadata {} --zone europe-west3-a --metadata-from-file ssh-keys={}'''

if (platform.system() == 'Linux'):
    keyDir = ('/home/jose-luis/.ssh/gotmKeys')

if (platform.system() == 'Windows'):
    keyDir = ('c:\\Users\\Guerrero\\Envs\\prognos_calibration\\prognos_calibration\\gotmKeys')
    

ip=''
instanceExists,ip = isInstance(instanceName)

if instanceExists:
    print('Instance {} is {}'.format(instanceName,ip) )

isStarted = False
if instanceExists and ip == 'TERMINATED' :
    callPopen('gcloud compute instances start {} --zone europe-west3-a'.format(instanceName))
    instanceExists,ip = isInstance(instanceName)
    isStarted = True
    print("Machine started and ip is {}".format(ip))

wasCreated=False
if not instanceExists and not isStarted:
    callPopen(createInstance.format(instanceName))
    wasCreated=True
    callPopen('rm -rf {}'.format(keyDir))
    callPopen('mkdir -p {}'.format(keyDir))
    generateSSHKey(username,keyDir)
    text_prepender('{}/{}.pub'.format(keyDir,username), '{}:'.format(username) )
    callPopen(addSSHKeys.format(instanceName,keyDir + '/{}.pub'.format(username)))
    callPopen('sed -i s/^{0}:// {1}/{0}.pub'.format(username,keyDir))
    ip=isInstance(instanceName)[1]
        #callPopen('chmod 400 {}'.format(keyDir))
        
print("The ip of {} is {}".format(instanceName,ip))

gcloud compute instances create gotm-cal --zone europe-west3-a --image-family debian-9 --image-project debian-cloud --machine-type n1-standard-2 
NAME      ZONE            MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
gotm-cal  europe-west3-a  n1-standard-2               10.156.0.3   35.198.127.198  RUNNING
Created [https://www.googleapis.com/compute/v1/projects/nivacatchment/zones/europe-west3-a/instances/gotm-cal].
rm -rf /home/jose-luis/.ssh/gotmKeys
mkdir -p /home/jose-luis/.ssh/gotmKeys
Successfully created key pair
gcloud compute instances add-metadata gotm-cal --zone europe-west3-a --metadata-from-file ssh-keys=/home/jose-luis/.ssh/gotmKeys/jose-luis.pub
Updated [https://www.googleapis.com/compute/v1/projects/nivacatchment/zones/europe-west3-a/instances/gotm-cal].
sed -i s/^jose-luis:// /home/jose-luis/.ssh/gotmKeys/jose-luis.pub
The ip of gotm-cal is 35.198.127.198


## Updating fabfile.py with credentials and ip

In [3]:
callPopen("sed -i s/^env\.hosts.*/env.hosts=\['{}']/ fabfile.py".format(ip))
callPopen("sed -i s/^env\.user.*/env.user=\'{}\'/ fabfile.py".format(username))
callPopen("sed -i s$^env\.key_filename.*$env\.key_filename='{}'$ fabfile.py".format(keyDir + '/' + username))
callPopen("sed -i s/^env\.roledefs.*/env.roledefs={{\\'{}\\':[\\'{}\\'],/ fabfile.py".format('stage',ip))

#Testing connection
#Adding key to remote machine
p = Popen("ssh -i {0}/{1} {1}@{2} -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no".format(keyDir,username,ip),shell=True,stdout=PIPE,stdin=PIPE)
print(p.communicate())

sed -i s/^env\.hosts.*/env.hosts=\['35.198.127.198']/ fabfile.py
sed -i s/^env\.user.*/env.user='jose-luis'/ fabfile.py
sed -i s$^env\.key_filename.*$env\.key_filename='/home/jose-luis/.ssh/gotmKeys/jose-luis'$ fabfile.py
sed -i s/^env\.roledefs.*/env.roledefs={\'stage\':[\'35.198.127.198\'],/ fabfile.py
(b'', None)


## Setting up GOTM and FABM in the remote machine

In [4]:
callPopen('fab testConnection')
callPopen('fab getUtilities')
callPopen('fab downloadModels') #note that the location of the key file is hardcoded in the getFABM function
callPopen('fab compileModels')
callPopen('fab getPROGNOS')
callPopen('fab testRun:/home/{}/PROGNOS/langtjern/langtjern.xml'.format(username))

fab testConnection
[35.198.127.198] Executing task 'testConnection'
[35.198.127.198] Executing task 'whoAmI'
[35.198.127.198] run: uname -a
[35.198.127.198] out: Linux gotm-cal 4.9.0-7-amd64 #1 SMP Debian 4.9.110-3+deb9u1 (2018-08-03) x86_64 GNU/Linux
[35.198.127.198] out: 

[35.198.127.198] run: whoami
[35.198.127.198] out: jose-luis
[35.198.127.198] out: 


Done.
Disconnecting from 35.198.127.198... done.
fab getUtilities
[35.198.127.198] Executing task 'getUtilities'
[35.198.127.198] Executing task 'update'
[35.198.127.198] Executing task 'updateMachine'
[35.198.127.198] run: sudo apt-get update
[35.198.127.198] out: 
[35.198.127.198] out: 0% [Working]
[35.198.127.198] out:             
[35.198.127.198] out: Ign:1 http://deb.debian.org/debian stretch InRelease
[35.198.127.198] out: 
[35.198.127.198] out: 0% [Connecting to prod.debian.map.fastly.net]
[35.198.127.198] out:                                              
[35.198.127.198] out: Get:2 http://deb.debian.org/debian stretch-up

[35.198.127.198] out: 0% [6 InRelease gpgv 6,377 B] [Waiting for headers]
[35.198.127.198] out:                                                    
[35.198.127.198] out: Hit:18 http://packages.cloud.google.com/apt google-cloud-packages-archive-keyring-stretch InRelease
[35.198.127.198] out: 
[35.198.127.198] out:                                                    
[35.198.127.198] out: 0% [6 InRelease gpgv 6,377 B]
[35.198.127.198] out:                              
[35.198.127.198] out: 0% [Working]
[35.198.127.198] out: 0% [16 InRelease gpgv 3,843 B] [Waiting for headers]
[35.198.127.198] out:                                                     
[35.198.127.198] out: Get:19 http://packages.cloud.google.com/apt cloud-sdk-stretch/main amd64 Packages [43.7 kB]
[35.198.127.198] out: 
[35.198.127.198] out:                                                     
[35.198.127.198] out: 0% [16 InRelease gpgv 3,843 B]
[35.198.127.198] out: 0% [19 Packages store 0 B] [16 InRelease gpgv 3,843 B]
[3

[35.198.127.198] out: Get:12 http://deb.debian.org/debian stretch/main amd64 cmake amd64 3.7.2-1 [3,038 kB]
[35.198.127.198] out: 
[35.198.127.198] out: 20% [12 cmake 26.1 kB/3,038 kB 1%]
[35.198.127.198] out:                                   
[35.198.127.198] out: 23% [Waiting for headers]
[35.198.127.198] out:                          
[35.198.127.198] out: Get:13 http://deb.debian.org/debian stretch/main amd64 libcurl3-gnutls amd64 7.52.1-5+deb9u6 [290 kB]
[35.198.127.198] out: 
[35.198.127.198] out: 23% [13 libcurl3-gnutls 65.5 kB/290 kB 23%]
[35.198.127.198] out:                                            
[35.198.127.198] out: 23% [Waiting for headers]
[35.198.127.198] out:                          
[35.198.127.198] out: Get:14 http://deb.debian.org/debian stretch/main amd64 liberror-perl all 0.17024-1 [26.9 kB]
[35.198.127.198] out: 
[35.198.127.198] out: 23% [14 liberror-perl 26.9 kB/26.9 kB 100%]
[35.198.127.198] out:                                            
[35.198.127.19

[35.198.127.198] out: 61% [44 g++-6 64.3 kB/7,094 kB 1%]
[35.198.127.198] out:                                   
[35.198.127.198] out: 68% [Waiting for headers]
[35.198.127.198] out:                          
[35.198.127.198] out: Get:45 http://deb.debian.org/debian stretch/main amd64 g++ amd64 4:6.3.0-4 [1,546 B]
[35.198.127.198] out: 
[35.198.127.198] out:                          
[35.198.127.198] out: Get:46 http://deb.debian.org/debian stretch/main amd64 libgfortran3 amd64 6.3.0-18+deb9u1 [265 kB]
[35.198.127.198] out: 
[35.198.127.198] out: 69% [46 libgfortran3 45.2 kB/265 kB 17%]
[35.198.127.198] out:                                         
[35.198.127.198] out: 69% [Waiting for headers]
[35.198.127.198] out:                          
[35.198.127.198] out: Get:47 http://deb.debian.org/debian stretch/main amd64 libgfortran-6-dev amd64 6.3.0-18+deb9u1 [299 kB]
[35.198.127.198] out: 
[35.198.127.198] out: 69% [47 libgfortran-6-dev 41.6 kB/299 kB 14%]
[35.198.127.198] out:        

[35.198.127.198] out: 
[35.198.127.198] out: Extracting templates from packages: 37%
[35.198.127.198] out: Extracting templates from packages: 74%
[35.198.127.198] out: Extracting templates from packages: 100%
[35.198.127.198] out: Selecting previously unselected package perl-modules-5.24.
[35.198.127.198] out: 
[35.198.127.198] out: (Reading database ... 
[35.198.127.198] out: (Reading database ... 5%
[35.198.127.198] out: (Reading database ... 10%
[35.198.127.198] out: (Reading database ... 15%
[35.198.127.198] out: (Reading database ... 20%
[35.198.127.198] out: (Reading database ... 25%
[35.198.127.198] out: (Reading database ... 30%
[35.198.127.198] out: (Reading database ... 35%
[35.198.127.198] out: (Reading database ... 40%
[35.198.127.198] out: (Reading database ... 45%
[35.198.127.198] out: (Reading database ... 50%
[35.198.127.198] out: (Reading database ... 55%
[35.198.127.198] out: (Reading database ... 60%
[35.198.127.198] out: (Reading database ... 65%
[35.198.127.198] o

...

[35.198.127.198] out: [ 75%] Building Fortran object CMakeFiles/meanflow.dir/meanflow/temperature.F90.o
[35.198.127.198] out: [ 76%] Building Fortran object CMakeFiles/meanflow.dir/meanflow/uequation.F90.o
[35.198.127.198] out: [ 76%] Building Fortran object CMakeFiles/meanflow.dir/meanflow/updategrid.F90.o
[35.198.127.198] out: [ 77%] Building Fortran object CMakeFiles/meanflow.dir/meanflow/vequation.F90.o
[35.198.127.198] out: [ 77%] Building Fortran object CMakeFiles/meanflow.dir/meanflow/water_balance.F90.o
[35.198.127.198] out: [ 78%] Building Fortran object CMakeFiles/meanflow.dir/meanflow/wequation.F90.o
[35.198.127.198] out: [ 78%] Linking Fortran static library libmeanflow.a
[35.198.127.198] out: [ 78%] Built target meanflow
[35.198.127.198] out: Scanning dependencies of target turbulence
[35.198.127.198] out: [ 80%] Building Fortran object CMakeFiles/turbulence.dir/turbulence/turbulence.F90.o
[35.198.127.198] out: [ 81%] Building Fortran object CMakeFiles/turbulence.dir/turbu

[35.198.127.198] out: Receiving objects:  15% (311/2070)   
[35.198.127.198] out: Receiving objects:  15% (315/2070), 1.38 MiB | 1.36 MiB/s   
[35.198.127.198] out: Receiving objects:  15% (325/2070), 7.41 MiB | 3.65 MiB/s   
[35.198.127.198] out: Receiving objects:  16% (332/2070), 7.41 MiB | 3.65 MiB/s   
[35.198.127.198] out: Receiving objects:  17% (352/2070), 7.41 MiB | 3.65 MiB/s   
[35.198.127.198] out: Receiving objects:  18% (373/2070), 7.41 MiB | 3.65 MiB/s   
[35.198.127.198] out: Receiving objects:  19% (394/2070), 15.50 MiB | 6.13 MiB/s   
[35.198.127.198] out: Receiving objects:  19% (413/2070), 26.31 MiB | 8.69 MiB/s   
[35.198.127.198] out: Receiving objects:  20% (414/2070), 26.31 MiB | 8.69 MiB/s   
[35.198.127.198] out: Receiving objects:  20% (427/2070), 44.24 MiB | 10.98 MiB/s   
[35.198.127.198] out: Receiving objects:  21% (435/2070), 44.24 MiB | 10.98 MiB/s   
[35.198.127.198] out: Receiving objects:  22% (456/2070), 44.24 MiB | 10.98 MiB/s   
[35.198.127.198] o

[35.198.127.198] out: Resolving deltas:  12% (131/1090)   
[35.198.127.198] out: Resolving deltas:  15% (167/1090)   
[35.198.127.198] out: Resolving deltas:  17% (187/1090)   
[35.198.127.198] out: Resolving deltas:  22% (250/1090)   
[35.198.127.198] out: Resolving deltas:  23% (257/1090)   
[35.198.127.198] out: Resolving deltas:  24% (262/1090)   
[35.198.127.198] out: Resolving deltas:  25% (283/1090)   
[35.198.127.198] out: Resolving deltas:  27% (299/1090)   
[35.198.127.198] out: Resolving deltas:  28% (306/1090)   
[35.198.127.198] out: Resolving deltas:  32% (354/1090)   
[35.198.127.198] out: Resolving deltas:  33% (364/1090)   
[35.198.127.198] out: Resolving deltas:  33% (370/1090)   
[35.198.127.198] out: Resolving deltas:  34% (374/1090)   
[35.198.127.198] out: Resolving deltas:  35% (382/1090)   
[35.198.127.198] out: Resolving deltas:  36% (393/1090)   
[35.198.127.198] out: Resolving deltas:  37% (406/1090)   
[35.198.127.198] out: Resolving deltas:  38% (415/1090) 

[35.198.127.198] Executing task 'testRun'
[35.198.127.198] run: xmlstarlet ed --inplace -u "scenario[@version='gotm-5.1']/@version" -v 'gotm-5.3' /home/jose-luis/PROGNOS/langtjern/langtjern.xml
[35.198.127.198] run: cd "$(dirname /home/jose-luis/PROGNOS/langtjern/langtjern.xml)" && editscenario --schemadir "$GOTMDIR"/schemas -e nml . langtjern.xml
[35.198.127.198] out: Node "gotm_fabm": number of children (2) is greater than the imposed maximum (1). Redundant child nodes will be deleted.
[35.198.127.198] out: Operating on values file with version gotm-5.3.
[35.198.127.198] out: Scenario validated successfully.
[35.198.127.198] out: Exporting values to namelist file(s) ....
[35.198.127.198] out: 

[35.198.127.198] run: cd "$(dirname "/home/jose-luis/PROGNOS/langtjern/langtjern.xml")" && gotm
[35.198.127.198] out:  ------------------------------------------------------------------------
[35.198.127.198] out:  GOTM started on 2018/08/08 at 11:54:41
[35.198.127.198] out:  -----------------

[35.198.127.198] out:           20 %
[35.198.127.198] out:           30 %
[35.198.127.198] out:           40 %
[35.198.127.198] out:           50 %
[35.198.127.198] out:           60 %
[35.198.127.198] out:           70 %
[35.198.127.198] out:           80 %
[35.198.127.198] out:           90 %
[35.198.127.198] out:          100 %
[35.198.127.198] out:  ------------------------------------------------------------------------
[35.198.127.198] out:     clean_up
[35.198.127.198] out:     clean_hypsograph
[35.198.127.198] out:         de-allocation hypsograph memory ...
[35.198.127.198] out:         done
[35.198.127.198] out:     clean_meanflow
[35.198.127.198] out:         de-allocation meanflow memory ...
[35.198.127.198] out:         done.
[35.198.127.198] out:     clean_turbulence
[35.198.127.198] out:         de-allocating turbulence memory ...
[35.198.127.198] out:         done.
[35.198.127.198] out:     clean_observations
[35.198.127.198] out:         de-allocate observation memory 